# Multi-fidelity Bayesian optimization (MFBO) with F3DASM

## 0. Goal of this notebook
This notebook serves to answer the following questions in order:
1. Which Python packages do I need to import in order to perform MFBO with GPR in F3DASM, and why?
2. What are the hyperparameters that MFBO with GPR uses and do I define them?
3. How do I run the MFBO algorithm?

## 1. Package dependencies

We first import the `f3dasm` library, which serves as the framework for doing BO with GPR.

In [ ]:
import f3dasm

The BO pipeline in F3DASM is based on the `pytorch` framework. As such, we need to import these packages:

In [ ]:
import torch, gpytorch

Lastly, we need to import the following packages to do the necessary intermediate math and visualization.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## 2. Define the hyperparameters

As with the [GPR in F3DASM](./f3dasm_sogpr.ipynb) tutorial, we assume that there is a data source defined by an analytic function. However, unlike the GPR tutorial, it is now necessary to define a `MultiFidelityFunction` in order to accomodate the multi-fidelity BO.

The main differences are:
- Each of the two fidelities has an associated function. In this tutorial, the low-fidelity function is an augmented version of the high-fidelity base function.
- Each of the two fidelity functions has an associated cost level: an assigned positive value which is incurred every time that the fidelity function in question is evaluated.

In [ ]:
dimensionality = 1
fidelity_parameters = [0.5, 1.]
costs = [0.5, 1.]

fun_class = f3dasm.functions.AlpineN2

base_fun = fun_class(
    dimensionality=dimensionality,
    scale_bounds=np.tile([0.0, 1.0], (dimensionality, 1)),
    )

###

fidelity_functions = []

for i, fidelity_parameter in enumerate(fidelity_parameters):

    fun = f3dasm.base.function.AugmentedFunction(
            base_fun=base_fun,
            fid=fidelity_parameter,
            dimensionality=base_fun.dimensionality,
            scale_bounds=base_fun.scale_bounds,
            )

    fidelity_functions.append(fun)

multifidelity_function = f3dasm.base.function.MultiFidelityFunction(
    fidelity_functions=fidelity_functions,
    fidelity_parameters=fidelity_parameters,
    costs=costs,
)

The goal is to minimize this objective function using Bayesian optimization.

### 2.1 MFGPR hyperparameters
In order for us to perform MFBO with GPR, we first need to specify the necessary hyperparameters of the multi-fidelity GP regression model; in this case, we use cokgj. 

This is done similarly in the way described in the [MFGPR tutorial](./f3dasm_mfgpr.ipynb), so the details will be omitted here.

In [ ]:
mean_module_list = torch.nn.ModuleList([
    gpytorch.means.ZeroMean(),
    gpytorch.means.ZeroMean()
])

covar_module_list = torch.nn.ModuleList([
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()),
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()),
])

likelihood = gpytorch.likelihoods.GaussianLikelihood()

opt_algo = torch.optim.Adam
opt_algo_kwargs = dict(lr=0.1)
training_iter = 50

noisy_data_bool = False
seed = 123

regressor_hyperparameters = f3dasm.machinelearning.gpr.Cokgj_Parameters(
    kernel=covar_module_list,
    mean=mean_module_list,
    likelihood=likelihood,
    noise_fix=1 - noisy_data_bool,
    opt_algo=opt_algo,
    opt_algo_kwargs=opt_algo_kwargs,
    training_iter=training_iter,
    )

regressor = f3dasm.machinelearning.Cokgj

### 2.2 MFBO hyperparameters

Next, hyperparameters for the BO scheme are to be selected. These consist of two main parts:

1. The regressor class and corresponding hyperparameters;
2. The acquisition function class and corresponding hyperparameters.

The regressor has been taken care of in the previous code cell. For this tutorial, we will select the variable fidelity upper confidence bound acquisition function [REFERENCE] with the appropriate hyperparameters:

In [ ]:
acquisition = f3dasm.base.acquisition.VFUpperConfidenceBound
acquisition_hyperparameters = f3dasm.optimization.bayesianoptimization_torch.Acquisition_Parameters(
    beta=0.4,
    maximize=False
)

In addition to the regressor and acquisition related objects, a few other parameters are to be defined:

In [ ]:
dimensionality = 1
iterations = 5
numbers_of_samples = [20, 5]
fidelity_parameters = [0.5, 1.]
costs = [0.5, 1.]
budget = 10
visualize_gp = True

We can now combine the two parts into the optimizer parameter object:

In [ ]:
opt_parameter = f3dasm.optimization.bayesianoptimization_torch.MFBayesianOptimizationTorch_Parameters(
    regressor=regressor,
    acquisition=acquisition,
    regressor_hyperparameters=regressor_hyperparameters,
    acquisition_hyperparameters=acquisition_hyperparameters,
    visualize_gp=visualize_gp,
)

## 3. Running MFBO

Before we can initialize the optimizer, we first need to identify the space and sampling method corresponding to the optimization problem.

In the case of a multi-fidelity Bayesian optimization problem, a sampler is associated with each fidelity:

In [ ]:
multifidelity_samplers = []

for i in [0, 1]:
    parameter_DesignSpace = f3dasm.make_nd_continuous_design(
        bounds=np.tile([0.0, 1.0], (dimensionality, 1)),
        dimensionality=dimensionality,
    )

    sampler = f3dasm.sampling.SobolSequence(design=parameter_DesignSpace, seed=seed)

    multifidelity_samplers.append(sampler)

Now, the optimizer can be defined with the appropriate parameters.

In [ ]:
optimizer = f3dasm.optimization.MFBayesianOptimizationTorch(
    data=f3dasm.ExperimentData(design=parameter_DesignSpace),
    multifidelity_function=multifidelity_function,
    )
optimizer.parameter = opt_parameter

In [ ]:
res = f3dasm.run_multi_fidelity_optimization(
    optimizer=optimizer,
    multifidelity_function=multifidelity_function,
    multifidelity_samplers=multifidelity_samplers,
    iterations=iterations,
    seed=seed,
    numbers_of_samples=numbers_of_samples,
    budget=budget
)


In [ ]:
res[0].data

In [ ]:
res[1].data